In [2]:
!python -m pip install --upgrade pip
!pip3 install scikit-build
!pip install cmake
!pip install --user opencv-contrib-python


  Obtaining dependency information for scikit-build from https://files.pythonhosted.org/packages/fa/af/b3ef8fe0bb96bf7308e1f9d196fc069f0c75d9c74cfaad851e418cc704f4/scikit_build-0.17.6-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/84.3 kB ? eta -:--:--
   ---------------------------------------- 84.3/84.3 kB 4.6 MB/s eta 0:00:00
  Obtaining dependency information for cmake from https://files.pythonhosted.org/packages/ff/21/f184a829edd2fe8edd844a607eb17f7bcc6e78e2a32f3b70238075060b76/cmake-3.27.6-py2.py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/34.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/34.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/34.6 MB 6.8 MB/s eta 0:00:06
   - -------------------------------------- 1.7/34.6 MB 17.6 MB/s eta 0:00:02
   ---- ----------------------------------- 3.7/34.6 MB 26.1 MB/s eta 0:00:02
   ------ --------------------------------- 5.2/34.6 MB 27.7 MB

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\iit80\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [1]:
!pip install opencv-python-headless==4.8.1.78
!pip install opencv-contrib-python==4.8.1.78

  Obtaining dependency information for opencv-python-headless==4.8.1.78 from https://files.pythonhosted.org/packages/e3/10/31b27a7473043eb5317f698ede00e7e129b2de378903bfe0bb4d785a7baf/opencv_python_headless-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.0 MB 991.0 kB/s eta 0:00:39
   ---------------------------------------- 0.4/38.0 MB 5.3 MB/s eta 0:00:08
    --------------------------------------- 0.6/38.0 MB 6.3 MB/s eta 0:00:06
    --------------------------------------- 0.6/38.0 MB 6.3 MB/s eta 0:00:06
   - -------------------------------------- 1.3/38.0 MB 6.2 MB/s eta 0:00:06
   - -------------------------------------- 1.7/38.0 MB 6.9 MB/s eta 0:00:06
   -- ------------------------------------- 2.0/38.0 MB 7.2 MB/s eta 0:00:05
   -- ------------------------------------- 2.3/38.0 MB 6.6 MB/s eta 0:00:06
   -- ------------------------------------- 2.5/38.0 MB 6.7 MB/

In [8]:
import cv2
import os
import numpy as np


얼굴 캡쳐

In [32]:
# 얼굴 검출을 위한 Haar cascades 분류기 로드
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)  # 웹캠 시작

# 이미지를 저장할 디렉토리 생성
save_folder = 'captured_faces'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

image_counter = 0
max_images = 200  # 총 찍을 이미지 개수

while image_counter < max_images:
    ret, frame = cap.read()  # 웹캠에서 프레임 읽기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 흑백으로 변환
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)  # 얼굴 검출

    for (x, y, w, h) in faces:
        # 얼굴 주위에 노란색 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
        
        # 얼굴 영역을 저장
        face_image = gray[y:y + h, x:x + w]
        image_filename = os.path.join(save_folder, f'face_{image_counter}.jpg')
        cv2.imwrite(image_filename, face_image)
        image_counter += 1

    cv2.imshow('WebCam', frame)  # 프레임 표시

    if cv2.waitKey(1) == ord('q'):  # 'q'키를 누르면 종료
        break

cap.release()  # 웹캠 해제
cv2.destroyAllWindows()  # 모든 창 닫기


모델 학습

In [33]:
# 함수: 폴더에서 이미지와 레이블을 로드
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            label = int(filename.split('_')[1].split('.')[0])  # 파일 이름에서 레이블 추출 (예: face_0.jpg -> 0)
            images.append(img)
            labels.append(label)
    return images, labels

# 얼굴 이미지가 저장된 폴더 경로
save_folder = 'captured_faces'

# LBPH 인식기 초기화
recognizer = cv2.face_LBPHFaceRecognizer.create()

# 이미지와 레이블 로드
faces, labels = load_images_from_folder(save_folder)

# 레이블을 np.array로 변환
labels = np.array(labels)

# 학습 수행
recognizer.train(faces, labels)

# 학습된 모델 저장
model_filename = 'face_recognition_model.yml'
recognizer.save(model_filename)

print("얼굴 인식 모델이 학습되었고 저장되었습니다.")

얼굴 인식 모델이 학습되었고 저장되었습니다.


얼굴 확인

In [35]:
# 저장된 얼굴 인식 모델 파일 경로
model_filename = 'face_recognition_model.yml'

# LBPH 인식기 초기화 및 학습된 모델 로드
recognizer = cv2.face_LBPHFaceRecognizer.create()
recognizer.read(model_filename)

# 얼굴 검출을 위한 Haar cascades 분류기 로드
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 웹캠 시작
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # 웹캠에서 프레임 읽기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 흑백으로 변환
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)  # 얼굴 검출

    for (x, y, w, h) in faces:
        # 얼굴 인식 수행
        face_roi = gray[y:y + h, x:x + w]
        label, confidence = recognizer.predict(face_roi)

        # 인식 결과에 따라 메시지와 사각형 색상 설정
        if confidence < 25:  # 얼굴 인식 정확도가 85 미만이면
            access_message = f"ACCESS GRANTED (Confidence: {round(100 - confidence, 2)}%)"
            box_color = (0, 255, 0)  # 초록색
        else:
            access_message = f"ACCESS DENIED (Confidence: {round(100 - confidence, 2)}%)"
            box_color = (0, 0, 255)  # 빨강색

        # 얼굴 주위에 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), box_color, 2)
        
        # 메시지를 화면에 표시
        cv2.putText(frame, access_message, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, box_color, 2)

    cv2.imshow('WebCam', frame)  # 프레임 표시

    if cv2.waitKey(1) == ord('q'):  # 'q'키를 누르면 종료
        break

cap.release()  # 웹캠 해제
cv2.destroyAllWindows()  # 모든 창 닫기

여러명 검사

얼굴 데이터 저장

In [21]:
# 얼굴 검출을 위한 Haar cascades 분류기 로드
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)  # 웹캠 시작

# 이미지를 저장할 디렉토리 생성
save_folder = 'captured_faces'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

image_counter = 0
max_images = 200  # 총 찍을 이미지 개수
person_name = input("Enter person's name and press Enter: ")


while image_counter < max_images:
    ret, frame = cap.read()  # 웹캠에서 프레임 읽기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 흑백으로 변환
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)  # 얼굴 검출

    for (x, y, w, h) in faces:
        # 얼굴 주위에 노란색 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
        
        # 얼굴 영역을 저장
        face_image = gray[y:y + h, x:x + w]
        
        # 각 사람의 이름을 입력 받음
        image_filename = os.path.join(save_folder, f'{person_name}_{image_counter}.jpg')
        cv2.imwrite(image_filename, face_image)
        image_counter += 1

    cv2.imshow('WebCam', frame)  # 프레임 표시

    if cv2.waitKey(1) == ord('q'):  # 'q'키를 누르면 종료
        break

cap.release()  # 웹캠 해제
cv2.destroyAllWindows()  # 모든 창 닫기

얼굴 학습과 라벨링

In [22]:
# 얼굴 검출을 위한 Haar cascades 분류기 로드
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 저장된 얼굴 이미지 로드 및 라벨링
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            label = filename.split('_')[0]  # 파일 이름에서 라벨 추출
            images.append(img)
            labels.append(label)
    return images, labels

# 얼굴 인식을 위한 라벨과 인덱스 매핑 생성
def create_label_mapping(labels):
    label_mapping = {}
    index = 0
    for label in labels:
        if label not in label_mapping:
            label_mapping[label] = index
            index += 1
    return label_mapping

# 얼굴 인식 모델 초기화 및 학습
recognizer = cv2.face.LBPHFaceRecognizer_create()
faces, labels = load_images_from_folder('captured_faces')
label_mapping = create_label_mapping(labels)

# 라벨을 인덱스로 변환하여 학습
label_indices = [label_mapping[label] for label in labels]
recognizer.train(faces, np.array(label_indices))

In [26]:
# 웹캠 시작
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # 웹캠에서 프레임 읽기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 흑백으로 변환
    faces = face_classifier.detectMultiScale(gray, 1.3, 10)  # 얼굴 검출

    for (x, y, w, h) in faces:
        # 얼굴 주위에 노란색 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
        
        # 얼굴 영역 추출
        face_roi = gray[y:y + h, x:x + w]

        # 얼굴 인식 수행
        label_index, confidence = recognizer.predict(face_roi)

        # 인식 결과에 따라 이름과 신뢰도 표시
        if confidence < 30:
            person_name = list(label_mapping.keys())[list(label_mapping.values()).index(label_index)]
            confidence_text = f"Confidence: {round(100 - confidence, 2)}%"
            result_text = f"Person: {person_name}, {confidence_text}"
            color = (0, 255, 0)  # 녹색 박스
        else:
            result_text = "Person: UNKNOWN"
            color = (0, 0, 255)  # 빨간색 박스

        # 메시지를 화면에 표시
        cv2.putText(frame, result_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    cv2.imshow('WebCam', frame)  # 프레임 표시

    if cv2.waitKey(1) == ord('q'):  # 'q'키를 누르면 종료
        break

cap.release()  # 웹캠 해제
cv2.destroyAllWindows()  # 모든 창 닫기